In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from freezegun import freeze_time

from src.email_manager import mock_send_email
from src.lambda_function import run_weekly_update_process, run_mid_week_update_process
from src.supabase_client import mock_upsert_training_week_update, mock_upsert_training_week_with_coaching
from src.supabase_client import list_users
from src.auth_manager import get_strava_client

In [3]:
def setup_mock_user():
    users = list_users()
    mock_user_id = int(os.environ["JAMIES_ATHLETE_ID"])

    mock_user = None
    for user in users:
        if user.athlete_id == mock_user_id:
            mock_user = user
            break

    print(f"Mock user: {mock_user}")
    return mock_user

In [4]:
mock_user = setup_mock_user()
get_strava_client(mock_user.athlete_id)

@freeze_time("2024-08-25 20:00:00")
def mock_weekly_update():
    run_weekly_update_process(
        user=mock_user,
        upsert_fn=mock_upsert_training_week_with_coaching,
        email_fn=mock_send_email,
    )

@freeze_time("2024-08-27 20:00:00")
def mock_mid_week_update():
    run_mid_week_update_process(
        user=mock_user,
        upsert_fn=mock_upsert_training_week_update,
        email_fn=mock_send_email,
    )

Mock user: athlete_id=98390356 email='voynow99@gmail.com' preferences='A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays' created_at=datetime.datetime(2024, 8, 25, 19, 46, 30, 851239, tzinfo=datetime.timezone.utc)
athlete_id=98390356 token still valid until 2024-08-28 04:51:52+00:00


In [7]:
mock_weekly_update()

athlete_id=98390356 token still valid until 2024-08-28 04:51:52+00:00
[TrainingSession(day=<Day.MON: 'mon'>, session_type=<SessionType.EASY: 'easy run'>, distance=5.0, notes='Easy pace to start the week relaxed'), TrainingSession(day=<Day.TUES: 'tues'>, session_type=<SessionType.EASY: 'easy run'>, distance=6.0, notes='Maintain easy pace; focusing on recovery and cadence'), TrainingSession(day=<Day.WED: 'wed'>, session_type=<SessionType.SPEED: 'speed workout'>, distance=6.0, notes='2 miles warm-up, 3x1mi at 10k pace with 400m recovery, 1 mile cool-down'), TrainingSession(day=<Day.FRI: 'fri'>, session_type=<SessionType.MODERATE: 'moderate run'>, distance=7.5, notes='Slightly faster than easy pace to build stamina'), TrainingSession(day=<Day.SAT: 'sat'>, session_type=<SessionType.LONG: 'long run'>, distance=16.5, notes='Focus on steady 9:37/mi pace, hydration during the run'), TrainingSession(day=<Day.SUN: 'sun'>, session_type=<SessionType.EASY: 'easy run'>, distance=6.5, notes='Super eas

In [8]:
mock_mid_week_update()

athlete_id=98390356 token still valid until 2024-08-28 04:51:52+00:00
{
    "athlete_id": 98390356,
    "activities": [
        {
            "date": "Monday, August 26, 2024",
            "distance_in_miles": 0.0,
            "elevation_gain_in_feet": 0.0,
            "pace_minutes_per_mile": NaN
        },
        {
            "date": "Tuesday, August 27, 2024",
            "distance_in_miles": 10.04,
            "elevation_gain_in_feet": 213.25,
            "pace_minutes_per_mile": 9.56
        }
    ],
    "training_week": [
        {
            "day": "mon",
            "session_type": "moderate run",
            "distance": 6.0,
            "notes": "Easy pace, focus on smooth form"
        },
        {
            "day": "wed",
            "session_type": "speed workout",
            "distance": 8.0,
            "notes": "4x1 mile @ 7m 30s pace with 400m recovery jogs"
        },
        {
            "day": "thurs",
            "session_type": "easy run",
            "distanc